In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import pandas as pd
import os
from PIL import Image
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Load CSV
df = pd.read_csv("/kaggle/input/skin-defects-acne-redness-and-bags-under-the-eyes/skin_defects.csv")

# Base image folder
base_dir = "/kaggle/input/skin-defects-acne-redness-and-bags-under-the-eyes/files"

images = []
labels = []

# Loop through each row and load front, left, and right images as separate samples
for idx, row in df.iterrows():
    label = row['type']
    
    for view in ['front', 'left_side', 'right_side']:
        img_rel_path = row[view].lstrip('/')  # remove starting slash
        img_full_path = os.path.join(base_dir, img_rel_path)

        try:
            img = Image.open(img_full_path).resize((128, 128)).convert('RGB')
            img = np.array(img) / 255.0
            images.append(img)
            labels.append(label)
        except Exception as e:
            print(f"Failed to load {img_full_path}: {e}")

# Convert to array
X = np.array(images)
y = LabelEncoder().fit_transform(labels)
y = to_categorical(y, num_classes=3)

# Train/validation split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print("Final dataset size:", X.shape)
print("Training set:", X_train.shape)
print("Validation set:", X_val.shape)


2025-07-17 07:32:10.819228: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752737531.196680      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752737531.308043      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Final dataset size: (90, 128, 128, 3)
Training set: (72, 128, 128, 3)
Validation set: (18, 128, 128, 3)


In [2]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam


In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=15,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
)

datagen.fit(X_train)

In [5]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
base_model.trainable = False  # Freeze base layers

# Add custom head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.4)(x)
output = Dense(3, activation='softmax')(x)  # 3 classes: acne, redness, bags

model = Model(inputs=base_model.input, outputs=output)

# Compile model
model.compile(optimizer=Adam(learning_rate=0.0005),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Optional callbacks
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ReduceLROnPlateau(patience=2, factor=0.5, verbose=1)
]

# Train the model
history = model.fit(
    datagen.flow(X_train, y_train, batch_size=4),
    validation_data=(X_val, y_val),
    epochs=30,
    callbacks=callbacks
)

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/30


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1752737634.548932     101 service.cc:148] XLA service 0x7ec88c0102f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1752737634.550651     101 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1752737634.550673     101 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1752737635.575891     101 cuda_dnn.cc:529] Loaded cuDNN version 90300


11/18 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.2966 - loss: 1.6886

I0000 00:00:1752737639.701049     101 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


18/18 ━━━━━━━━━━━━━━━━━━━━ 19s 342ms/step - accuracy: 0.3088 - loss: 1.6359 - val_accuracy: 0.4444 - val_loss: 0.9903 - learning_rate: 5.0000e-04
Epoch 2/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.5215 - loss: 1.1087 - val_accuracy: 0.5556 - val_loss: 0.8237 - learning_rate: 5.0000e-04
Epoch 3/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.6013 - loss: 0.8291 - val_accuracy: 0.8333 - val_loss: 0.6103 - learning_rate: 5.0000e-04
Epoch 4/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.7644 - loss: 0.5302 - val_accuracy: 0.6667 - val_loss: 0.6356 - learning_rate: 5.0000e-04
Epoch 5/30
16/18 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.5804 - loss: 0.8365
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.6020 - loss: 0.8068 - val_accuracy: 0.6667 - val_loss: 0.6911 - learning_rate: 5.0000e-04
Epoch 6/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.8840 - loss: 0.4906

In [6]:
loss, accuracy = model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.9444 - loss: 0.3167
Validation Accuracy: 94.44%


In [7]:
model.save("skin_concern_classifier_PT.h5")

In [8]:
import shutil

# Move the file to /kaggle/working so it's downloadable
shutil.move("skin_concern_classifier_PT.h5", "/kaggle/working/skin_concern_classifier_PT.h5")


'/kaggle/working/skin_concern_classifier_PT.h5'

In [9]:
loss, acc = model.evaluate(X_val, y_val)
print(f"Reloaded model accuracy: {acc * 100:.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.9444 - loss: 0.3167
Reloaded model accuracy: 94.44%
